In [2]:
from tkinter import *
from random import *
from tkinter import font
import pygame as pygame
pygame.init()

lignes = 20
collones = 10
zoom = 1
cote = 30
#/////////////////////////////////////////////////////////////////////////////////////////
li = lignes + 1
col = collones + 2
PLUS = cote*7
WIDTH = cote * col
HEIGHT = cote * li

if collones%2 == 0:
    x = WIDTH//2 - cote//2
else:
    x = WIDTH//2
y =- cote//2
   
my_font = "{Arial Black} 13 bold "

#////////////////////////////////////////////////////////////////////////////////////////
fen = Tk()

fen.title("TETRIS")
fen.resizable(False, False)

bloup = pygame.mixer.Sound("bloup.wav")
wouch = pygame.mixer.Sound("wouch.wav")
tetris = pygame.mixer.Sound("tetris.wav")
gameover = pygame.mixer.Sound("gameover.wav")
start = pygame.mixer.Sound("start.wav")
lvlup = pygame.mixer.Sound("lvlup.wav")
pygame.mixer.music.load("music.mp3")

score = 0
highscore = 0
level = 1
vitesse = 200 + 50*level 
r = 0

tetroO = [[x,y],[x+cote,y],[x,y+cote],[x+cote,y+cote],"O.gif"]
tetroI = [[x,y],[x,y+cote],[x,y+cote*2],[x,y+cote*3],"I.gif"]
tetroT = [[x,y+cote],[x+cote,y+cote],[x+cote*2,y+cote],[x+cote,y],"T.gif"]
tetroZ = [[x,y],[x+cote,y+cote],[x+cote,y],[x+cote*2,y+cote],"Z.gif"]
tetroS = [[x,y+cote],[x+cote,y],[x+cote,y+cote],[x+cote*2,y],"S.gif"]
tetroL = [[x,y],[x,y+cote],[x,y+cote*2],[x+cote,y+cote*2],"L.gif"]
tetroJ = [[x+cote,y],[x+cote,y+cote],[x+cote,y+cote*2],[x,y+cote*2],"J.gif"]

TETRO = [tetroO,tetroI,tetroT,tetroZ,tetroS,tetroL,tetroJ]

imageO = PhotoImage(master = fen, file = tetroO[4])
imageI = PhotoImage(master = fen, file = tetroI[4])
imageT = PhotoImage(master = fen, file = tetroT[4])
imageZ = PhotoImage(master = fen, file = tetroZ[4])
imageS = PhotoImage(master = fen, file = tetroS[4])
imageL = PhotoImage(master = fen, file = tetroL[4])
imageJ = PhotoImage(master = fen, file = tetroJ[4])

image_score = PhotoImage(master = fen, file = "score.gif")
image_highscore = PhotoImage(master = fen, file = "highscore.gif")
image_next = PhotoImage(master = fen, file = "next.gif")
image_lvl = PhotoImage(master = fen, file = "level.gif")

IMAGE = [imageO,imageI,imageT,imageZ,imageS,imageL,imageJ]

bord = PhotoImage (master = fen, file = "tab.gif")

#/////////////////////////////////////////////////////////////////////////////////////// 

# creation du terrain de jeu
def draw():
    global decor
    
    start.play()
    for i in range(100): 
        pygame.mixer.music.queue("music.mp3")
    pygame.mixer.music.set_volume(0.5)
    pygame.mixer.music.play()
    
    for i in range(li):
        decor = can.create_image(cote//2, cote//2+cote*i, image = bord, tags = "decor")
        decor = can.create_image(WIDTH-cote//2, cote//2+cote*i, image = bord, tags = "decor")
    for j in range (col):
        decor = can.create_image(cote//2+cote*j, HEIGHT-cote//2, image = bord, tags = "decor")
    for k in range (10):
        for l in range (lignes+1):
            r = randrange(7)
            deco = IMAGE[r]
            decor = can.create_image(WIDTH + cote//2 + cote*k , cote//2 + cote*l, image = deco)
    
    can.create_image(WIDTH + PLUS//2 + 1, HEIGHT - (HEIGHT - HEIGHT//2) - 5*zoom, image = image_score)
    can.create_image(WIDTH + PLUS//2 + 1, HEIGHT - (HEIGHT - HEIGHT//1.5) + 10*zoom, image = image_highscore)
    can.create_image(WIDTH + PLUS//2 + 1, HEIGHT - (HEIGHT//1.25), image = image_next)
    can.create_image(WIDTH + PLUS//2 + 1, HEIGHT - (HEIGHT//7.9), image = image_lvl)

#verification de la présence de ligne sur le terrain
def verif_ligne():
    
    global score
    cpt_tetris = 0
    
    for j in range (lignes):
        cpt = 0
        for i in range(collones):
            z = can.gettags(can.find_enclosed(cote + cote*i, cote*j, cote*2 + cote*i, cote + cote*j))
            if z != ():
                cpt = cpt + 1
            if cpt == collones:
                can.addtag_enclosed("ligne",cote,cote*j,cote + collones*cote,cote+cote*j)
                can.addtag_enclosed("pieceDown",cote, 0, cote + cote*collones,cote*j)
                can.delete("ligne")
                can.move("pieceDown",0,cote)
                can.dtag("pieceDown","pieceDown")
                cpt_tetris = cpt_tetris + 1
                score = score + 10
    if cpt_tetris == 4:
        tetris.play()
        score = score + 100
        
def verif_lvl():
    global level,vitesse
    level = (score // 200) + 1
    vitesse = 250 - (20 *(level - 1))

def update_text():
    fen.after(vitesse,update_text)
    v.set(str(score))
    w.set(str(highscore))
    w2.set(str(level))
                
def verif_gameover():
    for i in range (collones):
        z = can.gettags(can.find_enclosed(cote,0,cote +cote + cote*i,cote))
        if z != ():
            fen.after(1000,game_over())
        
def game_over():
    global score, highscore
    gameover.play()
    can.addtag_overlapping("pieceOver", cote, -120, cote + cote * collones, cote*lignes)
    can.delete("pieceOver")
    if score > highscore:
        highscore = score
    
    score = 0
                
#verif deplacement bas-gauche-droite
def verif_bas(piece):
    X = can.coords(piece)[0]
    Y = can.coords(piece)[1]
    z = can.gettags(can.find_enclosed(X + cote//2, Y + cote//2, X - cote//2, Y + cote+cote//2 ))
    if z == () or z == G or z == ('piece','current'):
        return True
    
def verif_bas2(piece):
    X = can.coords(piece)[0]
    Y = can.coords(piece)[1]
    z = can.gettags(can.find_enclosed(X +  cote//2, Y + cote + cote//2, X - cote//2, Y + cote*2 + cote//2 ))
    if z == () or z == G or z == ('piece','current'):
        return True
    
def verif_gauche(piece):
    X = can.coords(piece)[0]
    Y = can.coords(piece)[1]
    z = can.gettags(can.find_enclosed(X - cote + cote//2, Y - cote//2, X - cote - cote//2, Y + cote//2 ))
    
    if z == () or z == G or z == ('piece','current'):
        return True
    
def verif_droite(piece):
    X = can.coords(piece)[0]
    Y = can.coords(piece)[1]
    z = can.gettags(can.find_enclosed(X + cote + cote//2, Y - cote//2, X + cote//2, Y + cote//2 ))
   
    if z == () or z == G or z == ('piece','current'):
        return True
    
    
#deplacement bas-gauche-droite
def deplacement_bas():
    global preview0,preview1,preview2,preview3
    if verif_bas(piece0) == True and verif_bas(piece1) == True and verif_bas(piece2) == True and verif_bas(piece3) == True:
        can.move(piece0,0,cote)
        can.move(piece1,0,cote)
        can.move(piece2,0,cote)
        can.move(piece3,0,cote)
        fen.after(vitesse,deplacement_bas)

    elif verif_bas != True:
        can.addtag_enclosed("oldPreview",14*cote -10 ,3*cote,17*cote + 10 ,8*cote)
        can.delete("oldPreview")
        spawn_piece()  
    
    else :
        deplacement_bas()
        
def deplacement_rapide(event):
    if verif_bas2(piece0) == True and verif_bas2(piece1) == True and verif_bas2(piece2) == True and verif_bas2(piece3) == True:
        can.move(piece0,0,cote)
        can.move(piece1,0,cote)
        can.move(piece2,0,cote)
        can.move(piece3,0,cote)
    

def deplacement_gauche(event):
    if verif_gauche(piece0) == True and verif_gauche(piece1) == True and verif_gauche(piece2) == True and verif_gauche(piece3) == True:
        can.move(piece0,-cote,0)
        can.move(piece1,-cote,0)
        can.move(piece2,-cote,0)
        can.move(piece3,-cote,0)
        bloup.play()
    return True

def deplacement_droite(event):
    if verif_droite(piece0) == True and verif_droite(piece1) == True and verif_droite(piece2) == True and verif_droite(piece3) == True:
        can.move(piece0,cote,0)
        can.move(piece1,cote,0)
        can.move(piece2,cote,0)
        can.move(piece3,cote,0)
        bloup.play()
    return True


#verif rotation
def test_rotate(piece):
    
    j = can.gettags(decor)

    coord1 = can.coords(piece)[0]
    coord2 = can.coords(piece)[1]
    coords_centre = can.coords(piece1)
    
    if can.coords(piece) == [coords_centre[0], coords_centre[1] - cote]:
        z = can.gettags(can.find_enclosed(coord1 + 15*zoom, coord2 + 15*zoom, coord1 + 45*zoom, coord2 + 45*zoom))
        if z == () or z == G or z == ('piece','current'): 
            return True,[cote,cote]
                
    if can.coords(piece) == [coords_centre[0] + cote, coords_centre[1]]:
        z = can.gettags(can.find_enclosed(coord1 - 45*zoom, coord2 + 15*zoom, coord1 - 15*zoom, coord2 + 45*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[-cote,cote]
                                          
    if can.coords(piece) == [coords_centre[0], coords_centre[1] + cote]:
        z = can.gettags(can.find_enclosed(coord1 -45*zoom, coord2 - 45*zoom, coord1 - 15*zoom, coord2 - 15*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[-cote,-cote]
                                          
    if can.coords(piece) == [coords_centre[0] - cote , coords_centre[1]]:
        z = can.gettags(can.find_enclosed(coord1 + 15*zoom, coord2 - 45*zoom, coord1 + 45*zoom, coord2 - 15*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[cote,-cote]
                                
    if can.coords(piece) == [coords_centre[0] + cote, coords_centre[1] - cote]:
        z = can.gettags(can.find_enclosed(coord1 - 15*zoom, coord2 + 45*zoom, coord1 + 15*zoom, coord2 + 75*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[0,cote*2]
                                
    if can.coords(piece) == [coords_centre[0] + cote, coords_centre[1] + cote]:
        z = can.gettags(can.find_enclosed(coord1 - 75*zoom, coord2 - 15*zoom, coord1 - 45*zoom, coord2 + 15*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[-cote*2,0]
                                
    if can.coords(piece) == [coords_centre[0] - cote, coords_centre[1] + cote]:
        z = can.gettags(can.find_enclosed(coord1 - 15*zoom , coord2 - 75*zoom, coord1 + 15*zoom, coord2 - 45*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[0,-cote*2]
                                
    if can.coords(piece) == [coords_centre[0] - cote, coords_centre[1] - cote]:
        z = can.gettags(can.find_enclosed(coord1 + 45*zoom, coord2 - 15*zoom, coord1 + 75*zoom, coord2 + 15*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[cote*2,0]
                                                                        
    if can.coords(piece) == [coords_centre[0] + cote*2, coords_centre[1]]:
        z = can.gettags(can.find_enclosed(coord1 - 75*zoom, coord2 + 45*zoom, coord1 - 45*zoom, coord2 + 75*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[-cote*2,cote*2]
                                                                                
    if can.coords(piece) == [coords_centre[0], coords_centre[1]+ cote*2]:
        z = can.gettags(can.find_enclosed(coord1 - 75*zoom, coord2 - 75*zoom, coord1 - 45*zoom, coord2 - 45*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[-cote*2,-cote*2]
                                                                                        
    if can.coords(piece) == [coords_centre[0], coords_centre[1] - cote*2]:
        z = can.gettags(can.find_enclosed(coord1 + 45*zoom, coord2 -75*zoom, coord1 + 75*zoom, coord2 -45*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[cote*2,-cote*2]
                                                                                                
    if can.coords(piece) == [coords_centre[0] - cote*2, coords_centre[1]]:
        z = can.gettags(can.find_enclosed(coord1 + 45*zoom, coord2 + 45*zoom, coord1 + 75*zoom, coord2 + 75*zoom))
        if z == () or z == G or z == ('piece','current'):
            return True,[cote*2,cote*2]
    
        
def rotate(event):
    
    if test_rotate(piece0)[0] == True and test_rotate(piece2)[0] == True and test_rotate(piece3)[0] == True:
        can.move(piece0,test_rotate(piece0)[1][0],test_rotate(piece0)[1][1])
        can.move(piece2,test_rotate(piece2)[1][0],test_rotate(piece2)[1][1])
        can.move(piece3,test_rotate(piece3)[1][0],test_rotate(piece3)[1][1])
    wouch.play()
    return True


def spawn_piece():
    global piece0,piece1,piece2,piece3,image,G,r
    can.addtag_all("PieceMorte")
    verif_gameover()
    r1 = r
    
    tetro = TETRO[r1]
    image = IMAGE[r1]
   
    verif_ligne()
    verif_lvl()
    
    piece0 = can.create_image(tetro[0][0], tetro[0][1] - (3*cote), image = image, tags = "piece")
    piece1 = can.create_image(tetro[1][0], tetro[1][1] - (3*cote), image = image, tags = "piece")
    piece2 = can.create_image(tetro[2][0], tetro[2][1] - (3*cote), image = image, tags = "piece")
    piece3 = can.create_image(tetro[3][0], tetro[3][1] - (3*cote), image = image, tags = "piece")
    
    G = can.gettags(piece0)
    deplacement_bas()
    
    r = randrange(7)
    if r == r1:
        r = randrange(7)
    
    tetro_p = TETRO[r]
    image_p = IMAGE[r]
    
    preview0 = can.create_image(tetro_p[0][0] + (9*cote), tetro_p[0][1] + (4*cote + cote//2), image = image_p,)
    preview1 = can.create_image(tetro_p[1][0] + (9*cote), tetro_p[1][1] + (4*cote+(cote//2)), image = image_p,)
    preview2 = can.create_image(tetro_p[2][0] + (9*cote), tetro_p[2][1] + (4*cote+(cote//2)), image = image_p,)
    preview3 = can.create_image(tetro_p[3][0] + (9*cote), tetro_p[3][1] + (4*cote+(cote//2)), image = image_p,)
    
    
    
    
#//////////////////////////////////////////////////////////////////////////////////////
v = StringVar()
w = StringVar()
w2 = StringVar()

can = Canvas(fen,width = WIDTH + PLUS, height = HEIGHT, bg = "black")
Label_score = Label(fen, textvariable = v, fg='black', bg='white', font = my_font, height = 1, width = 8)
Label_highscore = Label(fen, textvariable = w, fg ='black', bg = 'white', font = my_font,height = 1, width = 8)
Label_level = Label(fen, textvariable = w2, fg ='black', bg = 'white', font = my_font,height = 1, width = 8)

fen.bind("<s>",deplacement_rapide)
fen.bind("<z>",rotate)
fen.bind("<d>",deplacement_droite)
fen.bind("<q>",deplacement_gauche)

update_text()
draw()
spawn_piece()

Label_score.place(x=414 ,y= 315)
Label_highscore.place(x=414, y=445)
Label_level.place(x = 414, y = 550)
can.pack()

fen.mainloop()